In [1]:
#import keras library
import keras
#import ImageDataGenerator class from keras
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
#Define the parameters /arguments for ImageDataGenerator class
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 rotation_range=180,
                                 zoom_range=0.2,
                                 horizontal_flip=True)


test_datagen=ImageDataGenerator(rescale=1./255)

In [28]:
#: Applying ImageDataGenerator functionality to trainset.
#give the path of training images folder
x_train = train_datagen.flow_from_directory(r'C:/Users/Lenovo/Downloads/forest-fire/Dataset/Dataset/train_set/',
                                            target_size = (128,128),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 436 images belonging to 2 classes.


In [27]:
#: Applying ImageDataGenerator functionality to testset.
#give the path of testing images folder
x_test = test_datagen.flow_from_directory(r'C:/Users/Lenovo/Downloads/forest-fire/Dataset/Dataset/test_set/',
                                          target_size = (128,128),
                                          batch_size = 32,
                                          class_mode = 'binary')

Found 121 images belonging to 2 classes.


In [29]:
'''import model building libraries'''

#To define linear intialisation import Sequential
from keras.models import Sequential
#To add layers import Dense
from keras.layers import Dense
#To create Convolution kernel import Convolution2D
from keras.layers import Convolution2D
#import Maxpooling layer
from keras.layers import MaxPooling2D
#import Flatten layer
from keras.layers import Flatten
import warnings
warnings.filterwarnings('ignore')

In [30]:
#intializing the model
model =Sequential()

In [31]:
#add convolutional layer
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))

In [32]:
#add maxpooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

In [33]:
#add flatten layer
model.add(Flatten())

In [34]:
#add hidden layer
model.add(Dense(output_dim=150,init='uniform',activation='relu'))

In [35]:
#add hidden layermodel.add(Dense(output_dim=64,init='uniform',activation='relu'))

In [36]:
#add output layer
model.add(Dense(output_dim=1,activation='sigmoid',init='uniform'))

In [37]:
#to check the indices
print(x_train.class_indices)

{'forest': 0, 'with fire': 1}


In [38]:
#configure the learning process
model.compile(loss = 'binary_crossentropy',
              optimizer = "adam",
              metrics = ["accuracy"])

In [39]:
#Training the model
model.fit_generator(x_train,steps_per_epoch=14,
                    epochs=10,validation_data=x_test,
                    validation_steps=4)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
14/14 [==============================] - 29s 2s/step - loss: 0.8843 - acc: 0.7035 - val_loss: 0.2716 - val_acc: 0.8512
Epoch 2/10
14/14 [==============================] - 26s 2s/step - loss: 0.3158 - acc: 0.8591 - val_loss: 0.1299 - val_acc: 0.9339
Epoch 3/10
14/14 [==============================] - 26s 2s/step - loss: 0.2065 - acc: 0.9272 - val_loss: 0.0839 - val_acc: 0.9669
Epoch 4/10
14/14 [==============================] - 26s 2s/step - loss: 0.1828 - acc: 0.9240 - val_loss: 0.1304 - val_acc: 0.9421
Epoch 5/10
14/14 [==============================] - 26s 2s/step - loss: 0.2387 - acc: 0.8966 - val_loss: 0.0828 - val_acc: 0.9752
Epoch 6/10
14/14 [==============================] - 26s 2s/step - loss: 0.2487 - acc: 0.8892 - val_loss: 0.0973 - val_acc: 0.9752
Epoch 7/10
14/14 [==============================] - 26s 2s/step - loss: 0.2381 - acc: 0.8725 - val_loss: 0.1006 - val_acc: 0.9587
Epoch 8/10
14/14 [========================

In [40]:
#save the model
model.save("forest1.h5")

# Prediction

In [41]:
#import load_model from keras.model
from keras.models import load_model
#import image class from keras
from keras.preprocessing import image
#import numpy
import numpy as np
#import cv2
import cv2

In [42]:
#load the saved model
model = load_model("forest1.h5")

In [46]:
#give any random image path
img = image.load_img(r'C:\Users\Lenovo\Downloads\forest-fire\Dataset\Dataset\test_set\with fire\19464620_401.jpg',target_size = (128,128))
x = image.img_to_array(img)
#expand the image shape
x = np.expand_dims(x,axis = 0)

In [47]:
pred = model.predict_classes(x)

In [48]:
pred

array([[1]])

# opencv prediction

In [49]:
cv2.__version__

'4.2.0'

In [ ]:
import cv2
#import facevec
import numpy as np
import smtplib
from keras.preprocessing import image 
from keras.models  import load_model
from twilio.rest import Client

model = load_model(r'C:\Users\Lenovo\Downloads\forest-fire\forest1.h5') 
video = cv2.VideoCapture(0)
name = ['forest','with fire']
    
while(1):
    success, frame = video.read()
    cv2.imwrite(r"C:\Users\Lenovo\Downloads\Detecting-Forest-Combustion-in-Forests-main\Detecting-Forest-Combustion-in-Forests-main\Training\image.jpg",frame)
    img = image.load_img(r"C:\Users\Lenovo\Downloads\Detecting-Forest-Combustion-in-Forests-main\Detecting-Forest-Combustion-in-Forests-main\Training\image.jpg",target_size = (64,64))
    x  = image.img_to_array(img)
    x = np.expand_dims(x,axis = 0)
    pred = model.predict_classes(x)
    p = pred[0]
    print(pred)
    cv2.putText(frame, "predicted  class = "+str(name[p]), (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1)
    
    
    pred = model.predict_classes(x)
    if pred[0]==1:
        account_sid = 'ACa56253bf3f2e2918bxxxxxxx'
        auth_token = 'a10cb957a1b8bc17abbxxxxxxx'
        client = Client(account_sid, auth_token)

        message = client.messages \
        .create(
         body='Forest Fire is detected, stay alert',
         from_=' +1503512xxxx', #twilio free number
         to='+919160xxxxxx')
        print(message.sid)
    
        print('Fire Detected')
        print ('SMS sent!')
        break
    else:
        print("no danger")
       #break
    cv2.imshow("image",frame)
   
    if cv2.waitKey(1) & 0xFF == ord('a'): 
        break

video.release()
cv2.destroyAllWindows()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no dange

no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no danger
[0]
no dan